In [1]:
# export
from dataclasses import dataclass
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from fastai.layers import conv_layer, NormType
from typing import *

In [2]:
# export
class Flatten(nn.Module):
    def forward(self, input): return input.view(input.size(0), -1)

class UnFlatten(nn.Module):
    def forward(self, input): return input.view(input.size(0), 64, 3, 3)

The following two methods are adapted from `fastai.layers.simple_cnn`.

In [3]:
# export
def ifnone(a, b): return a if (a is not None) else b

In [4]:
conv_layer?

In [5]:
def FeatureManipulator(): pass
# - just output the proper network
class Designer(): pass
# think about how I usually design a VAE, this class is essentially a mental model
# - shape of input data
# - design the network
#   - specify number of encoding dimensions
#   - add a layer, remove a layer, output_padding add or remove
#   - strides = 4, padding = 2 (for future updates)
#   - wish: dynamic feedback on the input shape and the output shape (where human inspection is needed)
#   - problem arises when: output of encoder is too small already, becomes zero or something like that
# - output the network to be trained, in several moving parts, maybe create a dummy class for them
# tools needed
# - IPy widgets
class Callback(): pass
class CallbackHandler(Callback): pass
class Trainer(CallbackHandler): pass

In [9]:
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
clear_output?
import time

In [ ]:
a = [1, 2, 3]
a.pop(-1)

In [10]:

from IPython.display import display
button = widgets.Button(description="Click Me!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("Button clicked.")

button.on_click(on_button_clicked)

Button(description='Click Me!', style=ButtonStyle())

Output()

In [11]:
len((1, 2, 3))

3

In [12]:
button = widgets.Button(description="Click Me!")
output = widgets.Output()



def on_button_clicked(b):
    with output:
        print("Button clicked.")
        
button.on_click(on_button_clicked)    
display(button, output)

Button(description='Click Me!', style=ButtonStyle())

Output()

In [13]:
def autorefresh(f):
    def inner():
        f()
        

In [14]:
widgets.IntSlider(description='(50% width, 80px height) button',
           layout=widgets.Layout(width='50%', height='80px'), orientation='vertical')

IntSlider(value=0, description='(50% width, 80px height) button', layout=Layout(height='80px', width='50%'), o…

In [15]:
list(zip(*[[1, 2, 3, ], [4, 5, 6]]))

[(1, 4), (2, 5), (3, 6)]

In [16]:
for (batch in zip(*[[1, 2, 3, ]]): print(batch)

SyntaxError: invalid syntax (<ipython-input-16-95dc100cc419>, line 1)

In [17]:
class Obj(): pass

In [18]:
a = Obj()

In [19]:
a

In [20]:
many_as = [a, a, a, a]
many_as_2 = [a, a, a, a]

def yield2(*args): yield args

def loop_through(list1, list2, yes=True):
    for i, j in zip(list1, list2): 
        if yes: yield2(i, j)
        else: print(i, j)

In [21]:
for i, j in loop_through(many_as, many_as_2):
    print(i == a)
    print(j == a)

TypeError: 'NoneType' object is not iterable

In [22]:
for _ in loop_through(many_as, many_as_2, yes=False): pass

<__main__.Obj object at 0x1a28545f28> <__main__.Obj object at 0x1a28545f28>
<__main__.Obj object at 0x1a28545f28> <__main__.Obj object at 0x1a28545f28>
<__main__.Obj object at 0x1a28545f28> <__main__.Obj object at 0x1a28545f28>
<__main__.Obj object at 0x1a28545f28> <__main__.Obj object at 0x1a28545f28>


TypeError: 'NoneType' object is not iterable

In [23]:
def print_1(a): print(a)

In [24]:
print_1(a)

In [25]:
layout = {
    'hbox_kernels': {}
}

In [26]:
from ipywidgets.widgets import *

In [27]:
def hbox(label:str=None, widgets:list=None):
    if label is not None: return HBox([Label(label)])
    else: return HBox(widgets)

In [28]:
def button(description:str, func) -> Button:
    button = widgets.Button(description=description)
    button.on_click(func)
    return button

In [29]:
def intslider(default, max, orientation='vertical'):
    w = widgets.IntSlider(
        value=default,
        min=0,
        max=max,
        step=1,
        description='',
        disabled=False,
        continuous_update=False,
        orientation=orientation,
        readout=True,
        readout_format='d',
    )
    w.layout.height='95%'
    return w

In [159]:
def output(): return Output()#layout={'border': '1px solid grey'})

In [160]:
import numpy as np

In [161]:
def fmap_shape_from_tuple(input_shape:tuple, out_channels:int, kernel_size:int, stride:int, padding:int, dilation:int=1, downsize=True, output_padding=0) -> tuple:
    """
    Compute the size of feature maps given the input size.

    param: input_size: a tuple with format (c, h, w)
    Reference:
    * https://pytorch.org/docs/stable/nn.html#conv2d
    * https://pytorch.org/docs/stable/nn.html#maxpool2d
    """
    if downsize:
        formula = lambda x : np.floor((x + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1)
    else:
        formula = lambda x : np.ceil((x - 1) * stride - 2 * padding + kernel_size + output_padding)

    h, w = input_shape[1], input_shape[2]
    h_out, w_out = formula(h), formula(w)
    shape_out = (int(out_channels), int(h_out), int(w_out))
    return shape_out

In [170]:
class VAEDesigner():

    def __init__(self, num_layers:int=3, input_shape=(3, 28, 28), downsize=True, paddings=None):
        
        self.num_layers = num_layers
        self.input_shape = input_shape
        self.downsize = downsize
        self.paddings = paddings
        
        assert (self.downsize and self.paddings is None) or (not self.downsize and self.paddings is not None),\
        print('Padding is only applicable to an upsampling network')
        
        if self.downsize: mode = 'DOWNSAMPLING'
        else: mode = 'UPSAMPLING'
        
        # hboxs of titles
        self.title_kernels  = hbox(label=f'Number of kernels of each {mode} layer')
        self.title_strides  = hbox(label=f'Size of strides of each {mode} layer')
        self.title_padding = hbox(label='Size of paddings for input arrays')
        self.title_shapes   = hbox(label=f'Output shape of each {mode} layer')        
        
        # sliders
        sliders_nkernels = [intslider(default=12, max=128) for _ in range(3)]
        sliders_skernels = [intslider(default=3, max=5) for _ in range(3)]
        sliders_strides  = [intslider(default=2, max=10) for _ in range(3)]
        sliders_paddings = [intslider(default=0, max=10, orientation='horizontal')]
        
        # hboxs of sliders
        self.hbox_nkernels = hbox(widgets=sliders_nkernels)
        self.hbox_skernels = hbox(widgets=sliders_skernels)
        self.hbox_strides = hbox(widgets=sliders_strides)
        self.hbox_padding = hbox(widgets=sliders_paddings)
        
        # buttons
        button_add_layer     = button(description='ADD LAYER', func=self.add_layer)
        button_remove_layer  = button(description='REMOVE LAYER', func=self.remove_layer)
        button_update_shapes = button(description='UPDATE SHAPES', func=self.update_output_widgets)
        self.hbox_buttons = hbox(widgets=[button_add_layer, button_remove_layer, button_update_shapes])
        
        self.hbox_shapes = hbox(widgets=[output() for i in range(self.num_layers)])
    
        self.update_layouts_for_hboxs()
        self.refresh_interface()
        

    # fundamental functions
    
    
    def cloop_thru_hboxs(self, 
                       hboxs_names:list, 
                       yield_per_step:bool=True, 
                       func=None):
        """
        Loop through widgets in several HBox's concurrently (hence the name cloop).
        
        For example, looping through [1, 2, 3] and [4, 5, 6] concurrently means to return 
        the following tuples on at a time (1, 4), (2, 5) and (3, 6).
        
        param: hboxs_names: the list of HBox's to be looped through
        param: yield_per_step: whether something need to be yielded at each iteration, default True
        param: func: if passed in, something is done (inplace) at each iteration, default None
        """
        
        def identity(i, w): pass
        func = ifnone(func, identity)

        list_of_tuples_of_children = [getattr(self, name, None).children for name in hboxs_names]
        assert None not in list_of_tuples_of_children, print('Children cannot be None.')

        for i, batch in enumerate(zip(*list_of_tuples_of_children)):
            if len(list_of_tuples_of_children) == 1: 
                func(i, batch[0])
                yield batch[0]
            else: 
                for w in batch: func(i, w) 
                yield batch
                
                
    def forpass(self, iterator): 
        """Loop through any generator without receiving its outputs; useful for inplace operations."""
        for _ in iterator: pass
                        
        
    def set_layout_for_box(self, 
                           box:HBox, 
                           height:str=None, 
                           align_items:str=None, 
                           justify_content:str=None) -> None:
        """
        Specify value for the `height`, `align_items` and `justify_items` attributes of a box.

        param: box: the box whose attributes will be setted
        param: height: the vertical height of the box in px, default None
        param: align_items: the location of widgets with respect to the horizontal midline of the box
        param: justify_content: the style by which widgets are organized horizontally
        """
        if height          is not None: box.layout.height = height
        if align_items     is not None: box.layout.align_items = align_items
        if justify_content is not None: box.layout.justify_content = justify_content
            
            
    # customizable functions
            
        
    def update_layouts_for_hboxs(self) -> None:
        for hbox in [self.hbox_nkernels, self.hbox_skernels, self.hbox_strides]: 
            self.set_layout_for_box(hbox, height='170px', align_items='center', justify_content='space-around')
        for hbox in [self.hbox_padding, self.hbox_shapes]:
            self.set_layout_for_box(hbox, height='30px', align_items='center', justify_content='space-around')

            
    def update_names_for_sliders(self):
        def set_name(i, w): w.description = 'L' + str(i)
        self.forpass(self.cloop_thru_hboxs(['hbox_nkernels', 'hbox_skernels', 'hbox_strides'], yield_per_step=False, func=set_name))
        
        
    def clear_output_widgets(self):
        for out in self.cloop_thru_hboxs(['hbox_shapes']): out.clear_output(wait=True)
        
        
    def update_output_widgets(self, b=None):
        self.clear_output_widgets()
        
        input_shape = self.input_shape
        padding = self.hbox_padding.children[0].value
        for i, (out, num_kernels, kernel_size, num_strides) in enumerate(self.cloop_thru_hboxs(['hbox_shapes', 'hbox_nkernels', 'hbox_skernels', 'hbox_strides'])):
            output_shape =  fmap_shape_from_tuple(
                input_shape=input_shape, 
                out_channels=num_kernels.value, 
                kernel_size=kernel_size.value, 
                stride=num_strides.value, 
                padding=0 if self.paddings is None else self.paddings[i], 
                downsize=self.downsize, 
            )
            with out: print(output_shape)
            input_shape = output_shape
#             padding = 0
              
            
    # functions activated by buttons
        
        
    def add_layer(self, b):
        if not self.num_layers > 7:
            self.hbox_nkernels.children += (intslider(default=12, max=64), )
            self.hbox_skernels.children += (intslider(default=2, max=5), )
            self.hbox_strides.children += (intslider(default=2, max=10), )
            self.hbox_shapes.children += (output(), )
            self.num_layers += 1
            self.refresh_interface()
        
        
    def remove_layer(self, b):
        if self.num_layers > 1:
            
            hbox_nkernels = list(self.hbox_nkernels.children)
            self.hbox_nkernels = hbox(widgets=hbox_nkernels[:-1])
            
            hbox_skernels = list(self.hbox_skernels.children)
            self.hbox_skernels = hbox(widgets=hbox_skernels[:-1])

            hbox_strides = list(self.hbox_strides.children)
            self.hbox_strides = hbox(widgets=hbox_strides[:-1])
            
            hbox_shapes = list(self.hbox_shapes.children)
            self.hbox_shapes = hbox(widgets=hbox_shapes[:-1])

            self.update_layouts_for_hboxs()
            
            self.num_layers -= 1
            self.refresh_interface()
        
        
    def refresh_interface(self):
        clear_output(wait=True)
        self.update_names_for_sliders()
        self.update_output_widgets()
        display(widgets.VBox([
                self.hbox_buttons,
                self.title_shapes,
                self.hbox_shapes,
#                 self.title_padding,
#                 self.hbox_padding,
                self.title_kernels, 
                self.hbox_nkernels,
                self.hbox_skernels,
                self.title_strides,
                self.hbox_strides,
        ]))

```python
self.decoder = nn.Sequential(
            UnFlatten(),
            nn.ConvTranspose2d(h_dim, 128, kernel_size=4, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128,64,kernel_size=4,stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, nc, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )
```

In [178]:
des = VAEDesigner(input_shape=(3, 28, 28), downsize=True)

In [176]:
des = VAEDesigner(input_shape=(64, 1, 1), downsize=False, paddings=(0, 1, 1))
#todo: organize this notebook, remove redundant things, copy this nb, make a another version of typing boxes instead of int sliders

In [148]:
def fmap_shape_from_tuple(input_shape:tuple, out_channels:int, kernel_size:int, stride:int, padding:int, dilation:int=1, downsize=True, output_padding=0) -> tuple:
    """
    Compute the size of feature maps given the input size.

    param: input_size: a tuple with format (c, h, w)
    Reference:
    * https://pytorch.org/docs/stable/nn.html#conv2d
    * https://pytorch.org/docs/stable/nn.html#maxpool2d
    """
    if downsize:
        formula = lambda x : np.floor((x + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1)
    else:
        formula = lambda x : np.ceil((x - 1) * stride - 2 * padding + kernel_size + output_padding)

    h, w = input_shape[1], input_shape[2]
    h_out, w_out = formula(h), formula(w)
    shape_out = (int(out_channels), int(h_out), int(w_out))
    return shape_out

In [136]:
fmap_shape_from_tuple((12, 4, 4), 64, 4, 2, padding=1, downsize=False)

(64, 8, 8)

In [137]:
fmap_shape_from_tuple((12, 8, 8), 64, 4, 2, padding=1, downsize=False)

(64, 16, 16)

In [120]:
 np.ceil((1 - 1) * 3)

0.0

In [610]:
print('next action: add stride and padding, have sensible defaults, add design a tool to compute shape dynamically, using a parser method')

next action: add stride and padding, have sensible defaults, add design a tool to compute shape dynamically, using a parser method


In [598]:
import asyncio

async def main():
    print('Hello ...')
    await asyncio.sleep(1)
    print('... World!')

main()

<coroutine object main at 0x1a261e4af0>

In [ ]:
a



In [310]:
w2 = widgets.BoundedIntText(
    value=7,
    min=0,
    max=10,
    step=1,
    description='L1 Kernels:',
    disabled=False
)

In [311]:
display(w1, w2)
widgets.jslink((w1, 'value'), (w2, 'value'))

IntSlider(value=5, continuous_update=False, description='L1 Kernels', max=5, orientation='vertical')

BoundedIntText(value=7, description='L1 Kernels:', max=10)

In [314]:
button = widgets.ToggleButton(
    value=False,
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

In [315]:
display(button)

ToggleButton(value=False, description='Click me', icon='check', tooltip='Description')

In [318]:
button.value

False

In [313]:
widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False
)

Checkbox(value=False, description='Check me')

In [319]:
items = [widgets.Label(str(i)) for i in range(4)]
widgets.HBox(items)

In [320]:
items = [widgets.Label(str(i)) for i in range(4)]
left_box = widgets.VBox([items[0], items[1]])
right_box = widgets.VBox([items[2], items[3]])
widgets.HBox([left_box, right_box])

In [287]:
w.value = 10990

In [289]:
w.keys

['_dom_classes',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_view_count',
 '_view_module',
 '_view_module_version',
 '_view_name',
 'continuous_update',
 'description',
 'description_tooltip',
 'disabled',
 'layout',
 'max',
 'min',
 'orientation',
 'readout',
 'readout_format',
 'step',
 'style',
 'value']

In [81]:
# export
@dataclass
class FeatureManipulator:
    actns:Collection[int]
    kernel_szs:Collection[int]=None
    strides:Collection[int]=None
    paddings:Collection[int]=None
    bn:bool=True
    decode:bool=False
    
    def __post_init__(self): self.seq = self._create()
        
    def _simplify_nested_sequential(self, seq:nn.Sequential) -> nn.Sequential:
        layers = []
        for child in seq.children():
            for layer in child:
                layers.append(layer)
        return nn.Sequential(*layers)
        
    def _create(self):
        nl = len(self.actns)-1
        kernel_szs = ifnone(self.kernel_szs, [4]*nl)
        strides    = ifnone(self.strides   , [2]*nl)
        paddings   = ifnone(self.paddings  , [1]*nl)
        if self.decode: kernel_szs, strides = map(lambda lst : list(reversed(lst)), [kernel_szs, strides])
        layers = [
            conv_layer(
                self.actns[i], 
                self.actns[i+1], 
                kernel_szs[i], 
                stride=strides[i], 
                padding=paddings[i],
                transpose=self.decode, 
                norm_type=(NormType.Batch if self.bn and i<(len(strides)-1) else None)) 
            for i in range(len(strides))
        ]
        # no adaptive average pooling here, since no predictions need to be made
        return self._simplify_nested_sequential(nn.Sequential(*layers))
    
    def __call__(self, images): return self.seq(images)
    
    def set_output_padding(self):
        assert self.decode, "Output padding is only needed for ConvTranspose2d layers."
        while True:
            
            print(fmap_shape_from_sequential(self.seq))
            layer_index = input('Layer to add output padding: ')
            self.seq[layer_index].output_padding = 1
            print(fmap_shape_from_sequential(self.seq))
            
            next_action = input('Undo (y or n): ')
            if next_action == 'y': self.seq[layer_index].output_padding = 0
            
            next_action = input('Continue (y or n): ')
            if next_action == 'y': return

In [82]:
fm = FeatureManipulator((3, 12, 12, 64), decode=True)

In [83]:
fm.set_output_padding()

NameError: name 'fmap_shape_from_sequential' is not defined

In [84]:
manipulator = feature_manipulator((3, 12, 12, 64), decode=False)

NameError: name 'feature_manipulator' is not defined

In [230]:
fmap_shape_from_sequential(manipulator, (3, 28, 28))

* Input shape: (3, 28, 28) -> Conv2d -> Output shape:  (12, 14, 14)
* Input shape: (12, 14, 14) -> Conv2d -> Output shape:  (12, 7, 7)
* Input shape: (12, 7, 7) -> Conv2d -> Output shape:  (64, 3, 3)


(64, 3, 3)

In [221]:
manipulator = feature_manipulator((3, 12, 12, 64), decode=True)

In [222]:
fmap_shape_from_sequential(manipulator, (64, 3, 3))

* Input shape: (64, 3, 3) -> ConvTranpose2d -> Output shape:  (12, 6, 6)
* Input shape: (12, 6, 6) -> ConvTranpose2d -> Output shape:  (12, 12, 12)
* Input shape: (12, 12, 12) -> ConvTranpose2d -> Output shape:  (64, 24, 24)


(64, 24, 24)

In [78]:
fmap_shape_from_sequential??

In [62]:
def simple_decoder():
    pass

In [8]:
import sys
sys.path.insert(0, '/Users/yangzhihan/code/python_modules/')
from path_magic import path_magic

In [9]:
from torch_utils.cnn import fmap_shape_from_sequential

In [66]:
@dataclass
class Encoder(nn.Module):
    actns: Collection[int]
    num_encoding_dims: int
    kernel_szs: Collection[int]=None
    strides: Collection[int]=None
    bn: bool=True
    num_encoding_dims: int=64
    
    def __post_init__(self):
        self.extract_features = feature_extractor()
        self.get_means = nn.Linear(h_dim, num_encoding_dims)
        self.get_logvars = nn.Linear(h_dim, num_encoding_dims)
        
    def forward(images):
        features = self.extract_features(images)
        return self.get_means(features), self.get_logvars(features)

In [67]:
Encoder?

In [16]:
class VAECallback():
    pass

In [17]:
class VAECallbackHandler(VAECallback):
    pass

In [19]:
class VAETrainer(VAECallbackHandler):
    pass

In [14]:
[3] * 5

[3, 3, 3, 3, 3]